# BDA - Project

Introducció:
- Storytelling
- Com fer la query? Quina? (volíem dades al llarg de molt temps, no d'un instant concret)
- covid té hashtag, vacuna no --> triar idiomes --> query #covid 24h ......

Data Exploration:
- Estudi de fields
- Quants són rt, originals, ..?
- Estudi de missing values i com estan codificats dels que ens afecten (text, geo)
- Explicar GEO!!!
- Distribució idiomes
- Llista / Mapa d'on estan els tweets
- Evolució del nb de tweets al llarg del temps (histograma..) (incloure notícies)
- Llista de top hashtags
- Llista top words (exclude stopwords,...)?

Estudi. Study on the acceptance of covid-19 vaccine:
- Mapa / Dist d'on es parla més de cada vacuna (mirar la distribució de la vacuna que es parla més segons el país)
- Evolució del sentiment al llarg del temps (relació amb notícies)
- Evolució del sentiment al llarg del temps de cada vacuna
- Sentiment segons pais. Mapa global de pro-vaccine i anti-vaccine
- Sentiment segons pais i vacuna?

Hagués estat bé tenir edat/sexe per fer estudi sociologic però no el tenim.

## 1. Problem description

## 2. Data gathering and storage

## 3. Data description

**Connection with the DB**

In [1]:
import json
import pymongo
from pymongo import MongoClient

client = pymongo.MongoClient('fpsds.synology.me', 27017, username='mongoadmin', password='bda')

# let's get the list of DBs already defined
print(client.list_database_names())

# Get the DB we want to use
tweets = client['tweets']['#covid_vaccine']

['admin', 'config', 'geo', 'local', 'tweets']


In [8]:
from bson.son import SON

Show an example of data sample

In [27]:
tweets.find_one({})

{'_id': 1334789658442665985,
 'created_at': 'Fri Dec 04 09:20:28 +0000 2020',
 'full_text': 'RT @21WIRE: UPDATE: Despite concerns about safety &amp; trial design, countries are already spending billions of dollars in taxpayer funds to p…',
 'display_text_range': [0, 144],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': '21WIRE',
    'name': 'Patrick Henningsen',
    'id': 16936686,
    'id_str': '16936686',
    'indices': [3, 10]}],
  'urls': []},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'geo': None,
 'coordinates': None,
 'place': None,
 'retweet_count': 25,
 'favorite_count': 0}

**Count the total number of tweets on the database and the number of tweets with geolocation**

In [16]:
print("Total number of tweets:", tweets.count_documents({}))

Total number of tweets: 68822


In [26]:
count_with_geo = tweets.count_documents({'$or': [
                                                {'geo': {'$ne': None}}, 
                                                {'coordinates': {'$ne': None}},
                                                {'place': {'$ne': None}}
                                        ]},{})
print("Number of tweets with geolocation:", count_with_geo)

Number of tweets with geolocation: 931


**Top 25 vaccine related hashtags**

In [37]:
from bson.son import SON

pipeline = [
        {'$unwind': "$entities.hashtags" },
        {'$group': {'_id': '$entities.hashtags.text', 'count': {'$sum': 1}}},
        {"$sort": SON([("count", pymongo.DESCENDING)])},
        {'$limit': 25}
    ]

res = tweets.aggregate(pipeline)
for sample in res:
    print(sample)

{'_id': 'vaccine', 'count': 42747}
{'_id': 'COVID19', 'count': 15302}
{'_id': 'Vaccine', 'count': 9458}
{'_id': 'coronavirus', 'count': 3461}
{'_id': 'COVID', 'count': 3160}
{'_id': 'Covid', 'count': 3068}
{'_id': 'Pfizer', 'count': 2682}
{'_id': 'Covid19', 'count': 2489}
{'_id': 'CovidVaccine', 'count': 2459}
{'_id': 'patents', 'count': 1854}
{'_id': 'COVIDー19', 'count': 1818}
{'_id': 'covid19', 'count': 1412}
{'_id': 'AstraZeneca', 'count': 1315}
{'_id': 'vaccination', 'count': 1207}
{'_id': 'covid', 'count': 1167}
{'_id': 'UK', 'count': 1123}
{'_id': 'BillGates', 'count': 962}
{'_id': 'pandemic', 'count': 794}
{'_id': 'Vaccination', 'count': 789}
{'_id': 'cdc', 'count': 783}
{'_id': 'Moderna', 'count': 777}
{'_id': 'vaccines', 'count': 766}
{'_id': 'GatesFoundation', 'count': 736}
{'_id': 'UN', 'count': 721}
{'_id': 'GAVI', 'count': 718}


In [2]:
r = tweets.count_documents({})
print(r)

721077


In [4]:
have_place = tweets.count_documents({'place': {'$ne': None}})
have_geo = tweets.count_documents({'geo': {'$ne': None}})
have_coordinates = tweets.count_documents({'coordinates': {'$ne': None}})
print(have_place, 100*have_place/r)
print(have_geo, 100*have_geo/r)
print(have_coordinates, 100*have_coordinates/r)

4980 0.6906335939157676
108 0.014977596012631106
108 0.014977596012631106


In [5]:
have_user_loc = tweets.count_documents({'user.location': {'$ne': ''}})
print(have_user_loc, 100*have_user_loc/r)

504936 70.02525389105463


In [6]:
have_my_geo = tweets.count_documents({'my_geo': {'$ne': None}})
print(have_my_geo, 100*have_my_geo/r)

353979 49.0903190643995


In [13]:
pip = [
    {'$group': {'_id': '$lang', 'count': {'$sum': 1}}},
    {'$sort': SON([('count', pymongo.DESCENDING)])},
    {'$limit': 10}
]
languages = tweets.aggregate(pip)
for lang in languages:
    print(lang)

{'_id': 'en', 'count': 531462}
{'_id': 'es', 'count': 106193}
{'_id': 'fr', 'count': 57837}
{'_id': 'und', 'count': 7199}
{'_id': 'hi', 'count': 3637}
{'_id': 'nl', 'count': 2285}
{'_id': 'ca', 'count': 1801}
{'_id': 'in', 'count': 1378}
{'_id': 'tl', 'count': 1190}
{'_id': 'pt', 'count': 963}


In [14]:
pip = [
    {'$match': {'my_geo': {'$ne': None}}},
    {'$group': {'_id': '$my_geo.country', 'count': {'$sum': 1}}},
    {'$sort': SON([('count', pymongo.DESCENDING)])},
    {'$limit': 10}
]
countries = tweets.aggregate(pip)
for country in countries:
    print(country)

{'_id': 'United States', 'count': 177805}
{'_id': 'United Kingdom', 'count': 32726}
{'_id': 'Canada', 'count': 24412}
{'_id': 'India', 'count': 17992}
{'_id': 'France', 'count': 16476}
{'_id': 'Mexico', 'count': 8866}
{'_id': 'Spain', 'count': 7249}
{'_id': 'Ecuador', 'count': 5053}
{'_id': 'Australia', 'count': 3793}
{'_id': 'Germany', 'count': 3237}
